# Angleichung der Spalten beider Datensätze

In [1]:
import os
import pandas as pd
import logging
import multiprocessing as mp

In [2]:
# Logging Parameter
logging.basicConfig(
    filename='21_angleichung_spalten.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Verzeichnis der CSV-Dateien
verzeichnis_ids17 = '../01_Datensaetze/improved_cic-ids-2017'
verzeichnis_ids18 = '../01_Datensaetze/improved_cse-cic-ids-2018'
nrows = 5 # None = alle Zeilen, ansonsten Anzahl der Zeilen für Testzwecke

## Entpacken der Datensätze

In [3]:
# ids17
!rm -r ../01_Datensaetze/improved_cic-ids-2017/*.csv
!unzip -o ../01_Datensaetze/improved_cic-ids-2017/CICIDS2017_improved.zip -d ../01_Datensaetze/improved_cic-ids-2017/

Archive:  ../01_Datensaetze/improved_cic-ids-2017/CICIDS2017_improved.zip
  inflating: ../01_Datensaetze/improved_cic-ids-2017/friday.csv  
  inflating: ../01_Datensaetze/improved_cic-ids-2017/monday.csv  
  inflating: ../01_Datensaetze/improved_cic-ids-2017/thursday.csv  
  inflating: ../01_Datensaetze/improved_cic-ids-2017/tuesday.csv  
  inflating: ../01_Datensaetze/improved_cic-ids-2017/wednesday.csv  


In [4]:
# ids18
!rm -r ../01_Datensaetze/improved_cse-cic-ids-2018/*.csv
!unzip -o ../01_Datensaetze/improved_cse-cic-ids-2018/CSECICIDS2018_improved.zip -d ../01_Datensaetze/improved_cse-cic-ids-2018/

Archive:  ../01_Datensaetze/improved_cse-cic-ids-2018/CSECICIDS2018_improved.zip
  inflating: ../01_Datensaetze/improved_cse-cic-ids-2018/Wednesday-14-02-2018.csv  
  inflating: ../01_Datensaetze/improved_cse-cic-ids-2018/Friday-16-02-2018.csv  
  inflating: ../01_Datensaetze/improved_cse-cic-ids-2018/Tuesday-20-02-2018.csv  
  inflating: ../01_Datensaetze/improved_cse-cic-ids-2018/Wednesday-21-02-2018.csv  
  inflating: ../01_Datensaetze/improved_cse-cic-ids-2018/Thursday-22-02-2018.csv  
  inflating: ../01_Datensaetze/improved_cse-cic-ids-2018/Friday-23-02-2018.csv  
  inflating: ../01_Datensaetze/improved_cse-cic-ids-2018/Wednesday-28-02-2018.csv  
  inflating: ../01_Datensaetze/improved_cse-cic-ids-2018/Thursday-01-03-2018.csv  
  inflating: ../01_Datensaetze/improved_cse-cic-ids-2018/Thursday-15-02-2018.csv  
  inflating: ../01_Datensaetze/improved_cse-cic-ids-2018/Friday-02-03-2018.csv  


## Prüfen der Spalten beider Datensätze

In [17]:
def count_columns(file):
    df = pd.read_csv(file, nrows=1)
    return {file:df.shape[1]}

def count_columns_in_directory(directory):
    try:
        logging.info(f"Start counting columns in directory {directory}")
        with mp.Pool(processes=5) as pool:
            result = pool.map(count_columns, [os.path.join(directory, datei) for datei in os.listdir(directory) if datei.endswith('.csv')])

        logging.info(f"Successfully counted columns in directory {directory}")
        return result
    except Exception as e:
        logging.error(f"Error counting columns in directory {directory}: {e}")
        return []

In [18]:
count_columns_in_directory(verzeichnis_ids17)

[{'../01_Datensaetze/improved_cic-ids-2017/tuesday.csv': 84},
 {'../01_Datensaetze/improved_cic-ids-2017/wednesday.csv': 84},
 {'../01_Datensaetze/improved_cic-ids-2017/friday.csv': 84},
 {'../01_Datensaetze/improved_cic-ids-2017/monday.csv': 84},
 {'../01_Datensaetze/improved_cic-ids-2017/thursday.csv': 84}]

In [19]:
count_columns_in_directory(verzeichnis_ids18)

[{'../01_Datensaetze/improved_cse-cic-ids-2018/Tuesday-20-02-2018.csv': 84},
 {'../01_Datensaetze/improved_cse-cic-ids-2018/Thursday-15-02-2018.csv': 84},
 {'../01_Datensaetze/improved_cse-cic-ids-2018/Thursday-01-03-2018.csv': 84},
 {'../01_Datensaetze/improved_cse-cic-ids-2018/Thursday-22-02-2018.csv': 84},
 {'../01_Datensaetze/improved_cse-cic-ids-2018/Wednesday-21-02-2018.csv': 84},
 {'../01_Datensaetze/improved_cse-cic-ids-2018/Wednesday-14-02-2018.csv': 84},
 {'../01_Datensaetze/improved_cse-cic-ids-2018/Friday-23-02-2018.csv': 84},
 {'../01_Datensaetze/improved_cse-cic-ids-2018/Friday-16-02-2018.csv': 84},
 {'../01_Datensaetze/improved_cse-cic-ids-2018/Friday-02-03-2018.csv': 84},
 {'../01_Datensaetze/improved_cse-cic-ids-2018/Wednesday-28-02-2018.csv': 84}]

Im Gegensatz zu den originalen Datensätzen passt hier die Spaltenanzahl.

## Angleichung der Spaltenbezeichnung

Zunächst prüfen, ob alle Header in den verschiedenen Dateien eines Datensatzes gleich sind.

In [20]:
# ids17
ids17_col_hash = []
for datei in os.listdir(verzeichnis_ids17):
    if datei.endswith('.csv'):
        pfad_zur_datei = os.path.join(verzeichnis_ids17, datei)
        df = pd.read_csv(pfad_zur_datei, nrows=nrows, skipinitialspace=True)
        ids17_col_hash.append(hash(tuple(df.columns)))

# Prüfen, ob alle Elemente in der Liste gleich sind
all_equal = all(x == ids17_col_hash[0] for x in ids17_col_hash)
print(f'Alle Elemente und somit alle Header in ids17 sind gleich:', all_equal)  

if all_equal:
    ids17_columns = df.columns

Alle Elemente und somit alle Header in ids17 sind gleich: True


In [21]:
ids18_col_hash = []
for datei in os.listdir(verzeichnis_ids18):
    if datei.endswith('.csv'):
        pfad_zur_datei = os.path.join(verzeichnis_ids18, datei)
        df = pd.read_csv(pfad_zur_datei, nrows=nrows)
        ids18_col_hash.append(hash(tuple(df.columns)))

# Prüfen, ob alle Elemente in der Liste gleich sind
all_equal = all(x == ids17_col_hash[0] for x in ids17_col_hash)
print(f'Alle Elemente und somit alle Header in ids18 sind gleich:', all_equal)  

if all_equal:
    ids18_columns = df.columns

Alle Elemente und somit alle Header in ids18 sind gleich: True


Prüfen, ob die Spaltennamen in beiden Datensätzen gleich sind

In [22]:
# Element für Element Vergleich
for col17, col18 in zip(ids17_columns, ids18_columns):
    if col17 == col18:
        print(f"Die Spalten sind gleich:".ljust(40), f'{col17}')
    else:
        print(f"Die Spalten sind unterschiedlich:".ljust(40), f"{col17} != {col18}")

Die Spalten sind gleich:                 Dst Port
Die Spalten sind gleich:                 Protocol
Die Spalten sind gleich:                 Flow Duration
Die Spalten sind gleich:                 Total Fwd Packet
Die Spalten sind gleich:                 Total Bwd packets
Die Spalten sind gleich:                 Total Length of Fwd Packet
Die Spalten sind gleich:                 Total Length of Bwd Packet
Die Spalten sind gleich:                 Fwd Packet Length Max
Die Spalten sind gleich:                 Fwd Packet Length Min
Die Spalten sind gleich:                 Fwd Packet Length Mean
Die Spalten sind gleich:                 Fwd Packet Length Std
Die Spalten sind gleich:                 Bwd Packet Length Max
Die Spalten sind gleich:                 Bwd Packet Length Min
Die Spalten sind gleich:                 Bwd Packet Length Mean
Die Spalten sind gleich:                 Bwd Packet Length Std
Die Spalten sind gleich:                 Flow Bytes/s
Die Spalten sind gleich:        

Da sich die Bezeichnung der Spalten zwischen ids17 und ids18 nicht wie in den originalen Datensätzen unterscheidet muss keine Anpassung erfolgen.

In [23]:
# Prüfen, ob alle Elemente in der Liste gleich sind
all_equal = all(x == ids17_col_hash[0] for x in ids17_col_hash)
print(f'Alle Header in ids17 und ids18 sind jetzt gleich:', all_equal)  

if all_equal:
    ids17_columns = df.columns

Alle Header in ids17 und ids18 sind jetzt gleich: True


In [24]:
del df

## Entfernen nicht relevanter Spalten

In [4]:
columns_to_remove = ['id', 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Timestamp', 'Attempted Category']

def remove_columns_from_file(file_path):
    temp_file = file_path + '.tmp'
    try:
        # Lade die CSV-Datei in Chunks
        chunksize = 500000  # Optional: Anpassbare Chunk-Größe
        chunks = pd.read_csv(file_path, chunksize=chunksize)

        # Verarbeite jeden Chunk und speichere in eine temporäre Datei
        for i, chunk in enumerate(chunks):
            # Entferne die spezifischen Spalten, falls sie vorhanden sind
            chunk = chunk.drop(columns=[col for col in columns_to_remove if col in chunk.columns], errors='ignore')

            # Speichere den Chunk in die temporäre Datei
            if i == 0:
                chunk.to_csv(temp_file, mode='w', header=True, index=False)
            else:
                chunk.to_csv(temp_file, mode='a', header=False, index=False)

        # Ersetze die Originaldatei durch die temporäre Datei
        os.replace(temp_file, file_path)

        logging.info(f"Datei verarbeitet und gespeichert: {file_path}")
    
    except Exception as e:
        logging.error(f"Fehler beim Verarbeiten der Datei {file_path}: {e}")
        if os.path.exists(temp_file):
            os.remove(temp_file)  # Entferne die temporäre Datei im Fehlerfall

def remove_columns_from_directory(directory):
    try:
        logging.info(f"Start processing directory {directory}")
        with mp.Pool(processes=5) as pool:
            pool.map(remove_columns_from_file, [os.path.join(directory, datei) for datei in os.listdir(directory) if datei.endswith('.csv')])

        logging.info(f"Successfully processed directory {directory}")
    except Exception as e:
        logging.error(f"Error processing directory {directory}: {e}")

In [5]:
remove_columns_from_directory(verzeichnis_ids17)

In [25]:
count_columns_in_directory(verzeichnis_ids17)

[{'../01_Datensaetze/improved_cic-ids-2017/tuesday.csv': 84},
 {'../01_Datensaetze/improved_cic-ids-2017/wednesday.csv': 84},
 {'../01_Datensaetze/improved_cic-ids-2017/friday.csv': 84},
 {'../01_Datensaetze/improved_cic-ids-2017/monday.csv': 84},
 {'../01_Datensaetze/improved_cic-ids-2017/thursday.csv': 84}]

In [7]:
remove_columns_from_directory(verzeichnis_ids18)

In [26]:
for datei in os.listdir(verzeichnis_ids18):
    if datei.endswith('.csv'):
        pfad_zur_datei = os.path.join(verzeichnis_ids18, datei)
        df = pd.read_csv(pfad_zur_datei, nrows=2)
        print(f"Datei: {datei.ljust(60)}", f"\tAnzahl der Spalten:", len(df.columns))

Datei: Tuesday-20-02-2018.csv                                       	Anzahl der Spalten: 84
Datei: Thursday-15-02-2018.csv                                      	Anzahl der Spalten: 84
Datei: Thursday-01-03-2018.csv                                      	Anzahl der Spalten: 84
Datei: Thursday-22-02-2018.csv                                      	Anzahl der Spalten: 84
Datei: Wednesday-21-02-2018.csv                                     	Anzahl der Spalten: 84
Datei: Wednesday-14-02-2018.csv                                     	Anzahl der Spalten: 84
Datei: Friday-23-02-2018.csv                                        	Anzahl der Spalten: 84
Datei: Friday-16-02-2018.csv                                        	Anzahl der Spalten: 84
Datei: Friday-02-03-2018.csv                                        	Anzahl der Spalten: 84
Datei: Wednesday-28-02-2018.csv                                     	Anzahl der Spalten: 84
